In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('..')

In [2]:
from langchain_chinese.document_loaders import LocalFilesLoader

In [3]:
fl = LocalFilesLoader()
fl.documents_folder

'/Users/xuehongwei/github/wencheng/data/documents'

In [4]:
fl.get_files()

['/Users/xuehongwei/github/wencheng/data/documents/gd_tax_arch.docx',
 '/Users/xuehongwei/github/wencheng/data/documents/autogpt-demo/求职简历.docx',
 '/Users/xuehongwei/github/wencheng/data/documents/autogpt-demo/供应商资格要求.pdf']

In [22]:
documents = fl.load()